## Diagnostics for simulate using a spline as the PSF.

### Configure environment

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/sa_diagnostics/simulate_spline_psf")
print(os.getcwd())

import numpy
numpy.random.seed(1)

In [ ]:
import storm_analysis.simulator.background as background
import storm_analysis.simulator.camera as camera
import storm_analysis.simulator.photophysics as photophysics
import storm_analysis.simulator.psf as psf
import storm_analysis.simulator.simulate as simulate

import storm_analysis.diagnostics.spliner.settings as settings
import storm_analysis.diagnostics.spliner.configure as configure
import storm_analysis.diagnostics.spliner.analyze_data as analyzeData
import storm_analysis.diagnostics.spliner.collate as collate


In [ ]:
settings.photons = [[20, 500], [20, 1000]]
print(settings.photons)

settings.iterations = 20
settings.n_frames = 10
settings.peak_locations = None
settings.smooth_psf = False


In [ ]:
# Make data using the spline for the PSF.

def makeData():
    index = 1
    for [bg, photons] in settings.photons:

        wdir = "test_{0:02d}".format(index)
        print(wdir)
        if not os.path.exists(wdir):
            os.makedirs(wdir)

        bg_f = lambda s, x, y, i3 : background.UniformBackground(s, x, y, i3, photons = bg)
        cam_f = lambda s, x, y, i3 : camera.Ideal(s, x, y, i3, settings.camera_offset)
        pp_f = lambda s, x, y, i3 : photophysics.AlwaysOn(s, x, y, i3, photons)
        psf_f = lambda s, x, y, i3 : psf.Spline(s, x, y, i3, settings.pixel_size, "psf.spline")
        
        sim = simulate.Simulate(background_factory = bg_f,
                                camera_factory = cam_f,
                                photophysics_factory = pp_f,
                                psf_factory = psf_f,
                                dither = False,
                                x_size = settings.x_size,
                                y_size = settings.y_size)
            
        sim.simulate(wdir + "/test.tif", "grid_list.hdf5", settings.n_frames)

        index += 1


### Configure

In [ ]:
configure.configure(False)

### Make Data

In [ ]:
makeData()

### Analyze data

In [ ]:
%time analyzeData.analyzeData()

### Collate data

In [ ]:
collate.collate()

### Reference results